# Laboratorium #1: zapytania do bazy SQL

## Zadanie #1: 

W bazie danych TRPrzychodnia zrobić zestawienie, które dla każdego dnia maja 2020 policzy liczbę odbytych wizyt, sumę opłat za wizyty oraz ilu lekarzy obsługiwało te wizyty.

In [ ]:
WITH dni_w_maju AS (
    SELECT CAST('2020-05-01' AS date) AS Dzien
    UNION ALL
    SELECT DATEADD(DAY, 1, Dzien)
    FROM dni_w_maju
    WHERE Dzien < '2020-05-31'
),
liczba_wizyt AS (
    SELECT 
        CAST(W.DataWizyty AS date) AS Dzien,
        COUNT(*) AS LiczbaWizyt,
        SUM(W.Oplata) AS SumaOplat,
        COUNT(DISTINCT W.IdLekarza) AS LiczbaLekarzy
    FROM [TRPrzychodnia].[dbo].[Wizyty] AS W
    WHERE W.DataWizyty >= '2020-05-01'
      AND W.DataWizyty  < '2020-06-01'
    GROUP BY CAST(W.DataWizyty AS date)
)
SELECT
    d.Dzien,
    COALESCE(v.LiczbaWizyt,   0) AS LiczbaWizyt,
    COALESCE(v.SumaOplat,     0) AS SumaOplat,
    COALESCE(v.LiczbaLekarzy, 0) AS LiczbaLekarzy
FROM dni_w_maju AS d
LEFT JOIN liczba_wizyt AS v
       ON d.Dzien = v.Dzien
ORDER BY d.Dzien; 

# Zadanie #2

W bazie danych TRPrzychodnia zrobić zestawienie zawierające następujące dane: nazwisko i imię lekarza , liczbę wizyt oraz sumę opłat za wizyty zrealizowane w 30 dniach, dla  których w roku 2021 osiągnięto największe  sumaryczne  dzienne wartości opłat za wizyty. W zestawieniu uwzględnić tych lekarzy którzy osiągnęli dwie największe wartości sumy opłat.

In [ ]:
-- na początku szukamy top-30 dni:

WITH wizyty_cte AS (
    SELECT
        CAST(W.DataWizyty AS date) AS DataWizyty,
        SUM(W.Oplata) AS SumaOplat
    FROM [TRPrzychodnia].[dbo].[Wizyty] AS W
    WHERE YEAR(W.DataWizyty) = 2021
    GROUP BY CAST(W.DataWizyty AS date)
),
top_wizyt as (SELECT TOP (30)
    w.DataWizyty
FROM wizyty_cte AS w
ORDER BY w.SumaOplat DESC)
select * from top_wizyt
order by DataWizyty desc;

In [ ]:
-- inner join żeby pobrać dane dot. lekarzy co przyjmowali  tego dnia
WITH wizyty_cte AS (
    SELECT
        CAST(W.DataWizyty AS date) AS DataWizyty,
        SUM(W.Oplata) AS SumaOplat
    FROM [TRPrzychodnia].[dbo].[Wizyty] AS W
    WHERE YEAR(W.DataWizyty) = 2021
    GROUP BY CAST(W.DataWizyty AS date)
),
top_wizyt as (SELECT TOP (30)
    w.DataWizyty
FROM wizyty_cte AS w
ORDER BY w.SumaOplat DESC),
top_lekarzy as (
	SELECT 
		w.IdLekarza,
		w.DataWizyty,
		COUNT(*) AS LiczbaWizyt,
		SUM(w.Oplata) AS SumaPerLekarz
	FROM [TRPrzychodnia].[dbo].[Wizyty] AS W
	INNER JOIN top_wizyt AS t
	ON CAST(w.DataWizyty AS date) = t.DataWizyty
	GROUP BY w.IdLekarza, w.DataWizyty
)
select * from top_lekarzy

In [ ]:
-- dense
WITH wizyty_cte AS (
    SELECT
        CAST(W.DataWizyty AS date) AS DataWizyty,
        SUM(W.Oplata) AS SumaOplat
    FROM [TRPrzychodnia].[dbo].[Wizyty] AS W
    WHERE YEAR(W.DataWizyty) = 2021
    GROUP BY CAST(W.DataWizyty AS date)
),
top_wizyt as (SELECT TOP (30)
    w.DataWizyty
FROM wizyty_cte AS w
ORDER BY w.SumaOplat DESC),
lekarze_wszyzcy as (
	SELECT 
		w.IdLekarza,
		w.DataWizyty,
		COUNT(*) AS LiczbaWizyt,
		SUM(w.Oplata) AS SumaPerLekarz
	FROM [TRPrzychodnia].[dbo].[Wizyty] AS W
	INNER JOIN top_wizyt AS t
	ON CAST(w.DataWizyty AS date) = t.DataWizyty
	GROUP BY w.IdLekarza, w.DataWizyty
),
top_lekarzy as (
	SELECT 
	     IdLekarza,
		 SUM(SumaPerLekarz) AS SumaOplat
	FROM lekarze_wszyzcy
	GROUP BY IdLekarza
)
SELECT TOP (2)
    topl.IdLekarza,
    lek.Nazwisko,
    lek.Imie,
    topl.SumaOplat
FROM top_lekarzy AS topl
LEFT JOIN [TRPrzychodnia].[dbo].[Lekarze]  AS lek
ON topl.IdLekarza = lek.IdLekarza
ORDER BY topl.SumaOplat DESC;

# Zadanie #3

W bazie danych TRUczelnia zrobić  zestawienie zawierające dane wykładowców(Nazwisko, imię i Nip), którzy wystawili  co najmniej 5 ocen przez trzy dni z rzędu.

In [ ]:
-- najpierw eliminujemy tych co nie wystawiali ocen 3 dni z rzędu
WITH oceny_per_dzien AS (
    SELECT 
        o.IdWykladowcy,
        CAST(o.DataOceny AS date) AS Dzien,
        COUNT(*) AS LiczbaOcen
    FROM [TRUczelnia].[dbo].[Oceny] AS o
    GROUP BY o.IdWykladowcy, CAST(o.DataOceny AS date)
),
dni_z_rzedu AS (
    SELECT
        IdWykladowcy,
        Dzien as Dzien3,
        LAG(Dzien, 1) OVER (PARTITION BY IdWykladowcy ORDER BY Dzien) AS Dzien2,
        LAG(Dzien, 2) OVER (PARTITION BY IdWykladowcy ORDER BY Dzien) AS Dzien1
    FROM oceny_per_dzien
)
SELECT * from dni_z_rzedu;

In [ ]:
-- wszyscy wykładowcy co wystawili >5 ocen w ciągu 3 dni
WITH oceny_per_dzien AS (
    SELECT 
        o.IdWykladowcy,
        CAST(o.DataOceny AS date) AS Dzien,
        COUNT(*) AS LiczbaOcen
    FROM [TRUczelnia].[dbo].[Oceny] AS o
    GROUP BY o.IdWykladowcy, CAST(o.DataOceny AS date)
),
dni_z_rzedu AS (
    SELECT
        IdWykladowcy,
        Dzien as Dzien3,
        LAG(Dzien, 1) OVER (PARTITION BY IdWykladowcy ORDER BY Dzien) AS Dzien2,
        LAG(Dzien, 2) OVER (PARTITION BY IdWykladowcy ORDER BY Dzien) AS Dzien1
    FROM oceny_per_dzien
),
suma_ocen as 
(SELECT
    dzr.IdWykladowcy,
    dzr.Dzien3,                      
    dzr.Dzien2,
    dzr.Dzien1,
    opdNow.LiczbaOcen   AS OcenyDniaAktualnego,
    opdPrev1.LiczbaOcen AS OcenyDniaPoprzedniego1,
    opdPrev2.LiczbaOcen AS OcenyDniaPoprzedniego2,
    (opdNow.LiczbaOcen + opdPrev1.LiczbaOcen + opdPrev2.LiczbaOcen) AS SumaOceny3Dni
FROM dni_z_rzedu AS dzr
JOIN oceny_per_dzien AS opdNow
    ON  opdNow.IdWykladowcy = dzr.IdWykladowcy
    AND opdNow.Dzien        = dzr.Dzien3
JOIN oceny_per_dzien AS opdPrev1
    ON  opdPrev1.IdWykladowcy = dzr.IdWykladowcy
    AND opdPrev1.Dzien        = dzr.Dzien2
JOIN oceny_per_dzien AS opdPrev2
    ON  opdPrev2.IdWykladowcy = dzr.IdWykladowcy
    AND opdPrev2.Dzien        = dzr.Dzien1
WHERE dzr.Dzien2 = DATEADD(DAY, -1, dzr.Dzien3)
  AND dzr.Dzien1 = DATEADD(DAY, -2, dzr.Dzien3))
select * from suma_ocen
where SumaOceny3Dni > 5;

In [ ]:
-- final touch
WITH oceny_per_dzien AS (
    SELECT 
        o.IdWykladowcy,
        CAST(o.DataOceny AS date) AS Dzien,
        COUNT(*) AS LiczbaOcen
    FROM [TRUczelnia].[dbo].[Oceny] AS o
    GROUP BY o.IdWykladowcy, CAST(o.DataOceny AS date)
),
dni_z_rzedu AS (
    SELECT
        IdWykladowcy,
        Dzien as Dzien3,
        LAG(Dzien, 1) OVER (PARTITION BY IdWykladowcy ORDER BY Dzien) AS Dzien2,
        LAG(Dzien, 2) OVER (PARTITION BY IdWykladowcy ORDER BY Dzien) AS Dzien1
    FROM oceny_per_dzien
),
suma_ocen as 
(SELECT
    dzr.IdWykladowcy,
    dzr.Dzien3,                      
    dzr.Dzien2,
    dzr.Dzien1,
    opdNow.LiczbaOcen   AS OcenyDniaAktualnego,
    opdPrev1.LiczbaOcen AS OcenyDniaPoprzedniego1,
    opdPrev2.LiczbaOcen AS OcenyDniaPoprzedniego2,
    (opdNow.LiczbaOcen + opdPrev1.LiczbaOcen + opdPrev2.LiczbaOcen) AS SumaOceny3Dni
FROM dni_z_rzedu AS dzr
JOIN oceny_per_dzien AS opdNow
    ON  opdNow.IdWykladowcy = dzr.IdWykladowcy
    AND opdNow.Dzien        = dzr.Dzien3
JOIN oceny_per_dzien AS opdPrev1
    ON  opdPrev1.IdWykladowcy = dzr.IdWykladowcy
    AND opdPrev1.Dzien        = dzr.Dzien2
JOIN oceny_per_dzien AS opdPrev2
    ON  opdPrev2.IdWykladowcy = dzr.IdWykladowcy
    AND opdPrev2.Dzien        = dzr.Dzien1
WHERE dzr.Dzien2 = DATEADD(DAY, -1, dzr.Dzien3)
  AND dzr.Dzien1 = DATEADD(DAY, -2, dzr.Dzien3))
SELECT DISTINCT wykl.IdWykladowcy,
	wykl.Imie,
	wykl.Nazwisko,
	wykl.Nip
FROM suma_ocen
JOIN [TRUczelnia].[dbo].[Wykladowcy] AS wykl
on suma_ocen.IdWykladowcy = wykl.IdWykladowcy
WHERE SumaOceny3Dni > 5;
